In [1]:
import tensorflow as tf
from tensorflow.data import Dataset
from ipynb.fs.full.model4 import Code2Model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import Mean
from torchvision import datasets, transforms
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import random
from tensorflow_addons.layers.normalizations import GroupNormalization

C:\Users\Rhea Pandita\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
checkpoint_callback = ModelCheckpoint("model_checkpoint.h5", save_best_only=True)
tf.keras.mixed_precision.set_global_policy('mixed_float16')

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [3]:
# Specify the number of training and testing samples you want to use
desired_train_size = 20000
desired_test_size = 10000

In [4]:
# Load the CIFAR-10 dataset
train_dataset = datasets.CIFAR10(
    root="data", train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor()
        #transforms.Resize((128, 128))  # Resize images to (128, 128)
    ])
)

Files already downloaded and verified


In [5]:
test_dataset = datasets.CIFAR10(
    root="data", train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor()
        #transforms.Resize((128, 128))  # Resize images to (128, 128)
    ])
)

Files already downloaded and verified


print(train_dataset.shape)
print(test_dataset.shape)

In [6]:
# Shuffle and split the training dataset into training and testing sets
train_size = desired_train_size
test_size = desired_test_size

In [7]:
# Create TensorFlow Dataset objects for training and testing
batch_size = 8
train_loader = Dataset.from_tensor_slices((train_dataset.data[:train_size], train_dataset.targets[:train_size])).batch(batch_size).shuffle(buffer_size=train_size)
test_loader = Dataset.from_tensor_slices((test_dataset.data[:test_size], test_dataset.targets[:test_size])).batch(batch_size)

In [8]:
input_layer = Input(shape=(32, 32, 3))
output_layer = Code2Model(32, 32, 16)(input_layer)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '{' was never closed (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '{' was never closed (<unknown>, line 1)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [9]:
model = Model(inputs=input_layer, outputs=output_layer)

In [10]:
optimizer = Adam(learning_rate=1e-4)
criterion = MeanSquaredError()
metric  = tf.keras.metrics.RootMeanSquaredError()

In [11]:
model.compile(optimizer=optimizer, loss=criterion, metrics=[metric],sample_weight_mode="temporal")

In [12]:
model.fit(train_loader, epochs=5, validation_data=test_loader,
         callbacks=[checkpoint_callback])

Epoch 1/5
2500/2500 [==============================] - 9147s 4s/step - loss: nan - root_mean_squared_error: nan - val_loss: inf - val_root_mean_squared_error: 4.7711
Epoch 2/5
2500/2500 [==============================] - 8824s 4s/step - loss: nan - root_mean_squared_error: nan - val_loss: inf - val_root_mean_squared_error: 4.7567
Epoch 3/5
2500/2500 [==============================] - 8834s 4s/step - loss: nan - root_mean_squared_error: nan - val_loss: inf - val_root_mean_squared_error: 4.7410
Epoch 4/5
2500/2500 [==============================] - 8961s 4s/step - loss: nan - root_mean_squared_error: nan - val_loss: inf - val_root_mean_squared_error: 4.7272
Epoch 5/5
2500/2500 [==============================] - 8810s 4s/step - loss: nan - root_mean_squared_error: nan - val_loss: inf - val_root_mean_squared_error: 4.7156


In [13]:
# Save the model as a .pt file
model.save("model.h5")

C:\Users\Rhea Pandita\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
